# Task 4 : Pipeline Hybride - Algorithmes + LLM

## Objectif
Combiner les **algorithmes classiques** de découverte de FDs avec l'**analyse sémantique par LLM** pour identifier les dépendances fonctionnelles **significatives**.

## Architecture du Pipeline

```
Dataset → [Algorithme FD] → FDs candidates → [LLM Sémantique] → FDs significatives
              ↓                                    ↓
         Validation                          Évaluation
         technique                           du sens
```

## Ce qu'on combine :
1. **Task 1** : Découverte algorithmique des FDs (précision technique)
2. **Task 2** : Analyse sémantique LLM (signification métier)
3. **Task 3** : Leçons sur l'échantillonnage (validation robuste)

## Résultat attendu
Un score hybride pour chaque FD : `Score = Validité_technique × Pertinence_sémantique`

In [ ]:
# Configuration et imports
import anthropic
import json
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from itertools import combinations
import time
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(override=True)

# Configurer Claude
CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

print("✅ Configuration chargée")

## 1. Charger les datasets

In [ ]:
# Charger les datasets
datasets = {}

# IRIS
iris_path = '../Datasets/iris/iris.data'
if os.path.exists(iris_path):
    datasets['iris'] = pd.read_csv(iris_path, header=None, 
                                    names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
    print(f"✅ iris: {datasets['iris'].shape}")

# BRIDGES
bridges_path = '../Datasets/pittsburgh+bridges/bridges.data.version1'
if os.path.exists(bridges_path):
    datasets['bridges'] = pd.read_csv(bridges_path, header=None,
                                       names=['IDENTIF', 'RIVER', 'LOCATION', 'ERECTED', 'PURPOSE', 
                                              'LENGTH', 'LANES', 'CLEAR-G', 'T-OR-D', 'MATERIAL',
                                              'SPAN', 'REL-L', 'TYPE'])
    print(f"✅ bridges: {datasets['bridges'].shape}")

# NURSERY
nursery_path = '../Datasets/nursery/nursery.data'
if os.path.exists(nursery_path):
    datasets['nursery'] = pd.read_csv(nursery_path, header=None,
                                       names=['parents', 'has_nurs', 'form', 'children', 
                                              'housing', 'finance', 'social', 'health', 'class'])
    print(f"✅ nursery: {datasets['nursery'].shape}")

print(f"\n📊 Datasets chargés: {list(datasets.keys())}")

## 2. Algorithme de découverte de FDs

On implémente un algorithme simplifié qui :
1. Teste toutes les paires (LHS, RHS) possibles
2. Calcule le taux de validité de chaque FD
3. Retourne les FDs avec un taux > seuil

In [ ]:
def check_fd(df, lhs_cols, rhs_col):
    """
    Vérifie si une FD tient dans un DataFrame.
    Retourne: (holds: bool, validity_rate: float, violations: int)
    """
    if isinstance(lhs_cols, str):
        lhs_cols = [lhs_cols]
    
    # Grouper par LHS et compter les valeurs uniques de RHS
    grouped = df.groupby(list(lhs_cols))[rhs_col].nunique()
    
    # Une FD tient si chaque groupe a exactement 1 valeur pour RHS
    violations = (grouped > 1).sum()
    total_groups = len(grouped)
    
    if total_groups == 0:
        return False, 0, 0
    
    validity_rate = ((total_groups - violations) / total_groups) * 100
    holds = violations == 0
    
    return holds, validity_rate, violations


def discover_fds(df, min_validity=100, max_lhs_size=2):
    """
    Découvre les FDs dans un DataFrame.
    
    Args:
        df: DataFrame à analyser
        min_validity: Taux minimum de validité (100 = FD exacte)
        max_lhs_size: Taille maximale du LHS
    
    Returns:
        Liste de FDs avec leurs statistiques
    """
    columns = list(df.columns)
    fds = []
    
    # Tester toutes les combinaisons de LHS
    for lhs_size in range(1, max_lhs_size + 1):
        for lhs_cols in combinations(columns, lhs_size):
            lhs_cols = list(lhs_cols)
            
            # Tester chaque RHS possible
            for rhs_col in columns:
                if rhs_col in lhs_cols:
                    continue  # RHS ne peut pas être dans LHS
                
                holds, validity, violations = check_fd(df, lhs_cols, rhs_col)
                
                if validity >= min_validity:
                    fds.append({
                        'lhs': lhs_cols,
                        'rhs': rhs_col,
                        'fd_string': f"{', '.join(lhs_cols)} -> {rhs_col}",
                        'holds': holds,
                        'validity': validity,
                        'violations': violations,
                        'lhs_size': len(lhs_cols)
                    })
    
    return fds

print("✅ Fonctions check_fd() et discover_fds() définies")

## 3. Découvrir les FDs candidates

In [ ]:
# Découvrir les FDs pour chaque dataset
# On cherche les FDs APPROXIMATIVES (>= 90% de validité) pour avoir plus de résultats intéressants

all_discovered_fds = {}

print("="*80)
print("DÉCOUVERTE ALGORITHMIQUE DES FDs (exactes ET approximatives)")
print("="*80)

for name, df in datasets.items():
    print(f"\n🔍 Analyse de {name}...")
    
    # Découvrir les FDs avec au moins 90% de validité
    fds = discover_fds(df, min_validity=90, max_lhs_size=2)
    
    # Trier par validité décroissante
    fds = sorted(fds, key=lambda x: (-x['validity'], x['lhs_size']))
    
    # Garder les 15 plus intéressantes
    fds_filtered = fds[:15]
    
    all_discovered_fds[name] = fds_filtered
    
    print(f"   ✅ {len(fds)} FDs trouvées (>= 90% validité)")
    print(f"   📋 Top FDs:")
    for fd in fds_filtered[:5]:
        status = "✓ 100%" if fd['validity'] == 100 else f"~{fd['validity']:.0f}%"
        print(f"      - {fd['fd_string']} [{status}]")

print(f"\n📊 Total: {sum(len(fds) for fds in all_discovered_fds.values())} FDs candidates")

## 4. Analyse sémantique par LLM

Le LLM évalue chaque FD sur plusieurs critères :
- **Signification** : La FD a-t-elle un sens métier ?
- **Utilité** : Est-elle utile pour comprendre les données ?
- **Type** : Clé primaire, règle métier, accidentelle, etc.

In [ ]:
def evaluate_fd_semantically(fd_string, dataset_name, columns_info):
    """
    Utilise le LLM pour évaluer sémantiquement une FD.
    Retourne un score de 0 à 10 et une catégorie.
    """
    prompt = f"""Tu es un expert en bases de données et en qualité des données.

Dataset: {dataset_name}
Colonnes disponibles: {columns_info}

Évalue cette dépendance fonctionnelle: {fd_string}

Réponds UNIQUEMENT avec ce format JSON (pas d'autre texte):
{{
    "score": <nombre de 0 à 10>,
    "category": "<une parmi: key, business_rule, derived, accidental, meaningless>",
    "reason": "<explication courte en 10 mots max>"
}}

Critères:
- score 8-10: FD significative (règle métier, clé naturelle)
- score 5-7: FD utile mais pas fondamentale
- score 2-4: FD technique ou dérivée
- score 0-1: FD accidentelle ou sans sens

Categories:
- key: identifiant unique ou clé primaire
- business_rule: règle métier logique
- derived: attribut calculé/dérivé
- accidental: corrélation sans causalité
- meaningless: pas de sens sémantique"""
    
    try:
        message = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=200,
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = message.content[0].text.strip()
        
        # Parser le JSON
        # Nettoyer si nécessaire
        if response_text.startswith("```"):
            response_text = response_text.split("```")[1]
            if response_text.startswith("json"):
                response_text = response_text[4:]
        
        result = json.loads(response_text)
        return result
    except Exception as e:
        return {"score": 5, "category": "unknown", "reason": f"Erreur: {str(e)[:30]}"}

print("✅ Fonction d'évaluation sémantique définie")

## 5. Pipeline Hybride Complet

In [ ]:
# Exécuter le pipeline hybride sur toutes les FDs découvertes

hybrid_results = []

print("="*80)
print("PIPELINE HYBRIDE : Algorithme + LLM")
print("="*80)

for dataset_name, fds in all_discovered_fds.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'='*60}")
    
    columns_info = list(datasets[dataset_name].columns)
    
    for fd in fds[:10]:  # Limiter à 10 FDs par dataset
        print(f"\n🔄 Évaluation: {fd['fd_string']}")
        
        # Évaluation sémantique par LLM
        semantic_eval = evaluate_fd_semantically(fd['fd_string'], dataset_name, columns_info)
        
        # Calculer le score hybride
        technical_score = fd['validity'] / 10  # 0-10
        semantic_score = semantic_eval.get('score', 5)
        hybrid_score = (technical_score + semantic_score) / 2
        
        result = {
            'dataset': dataset_name,
            'fd': fd['fd_string'],
            'technical_validity': fd['validity'],
            'semantic_score': semantic_score,
            'hybrid_score': hybrid_score,
            'category': semantic_eval.get('category', 'unknown'),
            'reason': semantic_eval.get('reason', '')
        }
        hybrid_results.append(result)
        
        # Afficher le résultat
        print(f"   📊 Technique: {fd['validity']:.0f}% | Sémantique: {semantic_score}/10 | Hybride: {hybrid_score:.1f}")
        print(f"   🏷️ Catégorie: {semantic_eval.get('category', '?')}")
        print(f"   💬 Raison: {semantic_eval.get('reason', '?')}")
        
        time.sleep(1)  # Pause pour l'API

print("\n✅ Pipeline hybride terminé !")

## 6. Tableau des résultats

In [ ]:
# Créer le tableau des résultats
results_df = pd.DataFrame(hybrid_results)

print("\n" + "="*100)
print("TABLEAU DES RÉSULTATS HYBRIDES")
print("="*100)

# Trier par score hybride décroissant
results_df = results_df.sort_values('hybrid_score', ascending=False)

display(results_df[['dataset', 'fd', 'technical_validity', 'semantic_score', 'hybrid_score', 'category']])

## 7. Classification des FDs

In [ ]:
# Classifier les FDs par catégorie

print("\n" + "="*80)
print("CLASSIFICATION DES FDs PAR CATÉGORIE")
print("="*80)

# FDs significatives (score hybride >= 7)
significant = results_df[results_df['hybrid_score'] >= 7]
print(f"\n🌟 FDs SIGNIFICATIVES (score >= 7): {len(significant)}")
for _, row in significant.iterrows():
    print(f"   [{row['dataset']}] {row['fd']} (score: {row['hybrid_score']:.1f}, {row['category']})")

# FDs utiles (score entre 5 et 7)
useful = results_df[(results_df['hybrid_score'] >= 5) & (results_df['hybrid_score'] < 7)]
print(f"\n👍 FDs UTILES (5 <= score < 7): {len(useful)}")
for _, row in useful.iterrows():
    print(f"   [{row['dataset']}] {row['fd']} (score: {row['hybrid_score']:.1f}, {row['category']})")

# FDs à ignorer (score < 5)
ignore = results_df[results_df['hybrid_score'] < 5]
print(f"\n❌ FDs À IGNORER (score < 5): {len(ignore)}")
for _, row in ignore.iterrows():
    print(f"   [{row['dataset']}] {row['fd']} (score: {row['hybrid_score']:.1f}, {row['category']})")

# Statistiques par catégorie
print("\n" + "="*80)
print("STATISTIQUES PAR CATÉGORIE")
print("="*80)
category_stats = results_df.groupby('category').agg({
    'fd': 'count',
    'hybrid_score': 'mean'
}).rename(columns={'fd': 'count', 'hybrid_score': 'avg_score'})
display(category_stats.sort_values('avg_score', ascending=False))

## 8. Visualisation

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Distribution des scores hybrides
axes[0].hist(results_df['hybrid_score'], bins=10, edgecolor='black', color='steelblue')
axes[0].set_xlabel('Score Hybride')
axes[0].set_ylabel('Nombre de FDs')
axes[0].set_title('Distribution des Scores Hybrides')
axes[0].axvline(x=7, color='green', linestyle='--', label='Seuil significatif')
axes[0].axvline(x=5, color='orange', linestyle='--', label='Seuil utile')
axes[0].legend()

# 2. Score technique vs sémantique
colors = {'key': 'green', 'business_rule': 'blue', 'derived': 'orange', 
          'accidental': 'red', 'meaningless': 'gray', 'unknown': 'purple'}
for cat in results_df['category'].unique():
    subset = results_df[results_df['category'] == cat]
    axes[1].scatter(subset['technical_validity'], subset['semantic_score'], 
                    label=cat, c=colors.get(cat, 'black'), s=100, alpha=0.7)
axes[1].set_xlabel('Validité Technique (%)')
axes[1].set_ylabel('Score Sémantique (0-10)')
axes[1].set_title('Technique vs Sémantique')
axes[1].legend()

# 3. Répartition par catégorie
category_counts = results_df['category'].value_counts()
axes[2].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
            colors=[colors.get(c, 'gray') for c in category_counts.index])
axes[2].set_title('Répartition par Catégorie')

plt.tight_layout()
plt.savefig('../results/task4_hybrid_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualisation sauvegardée: results/task4_hybrid_analysis.png")

## 9. Sauvegarder les résultats

In [ ]:
# Sauvegarder les résultats
results_df.to_csv('../results/task4_hybrid_results.csv', index=False)
print("✅ Résultats sauvegardés: results/task4_hybrid_results.csv")

# Sauvegarder en JSON aussi
with open('../results/task4_hybrid_results.json', 'w', encoding='utf-8') as f:
    json.dump(hybrid_results, f, indent=2, ensure_ascii=False)
print("✅ Résultats JSON: results/task4_hybrid_results.json")

## 10. Conclusion et Synthèse

### Avantages du Pipeline Hybride

| Approche | Forces | Faiblesses |
|----------|--------|------------|
| **Algorithme seul** | Précis, exhaustif | Trouve des FDs sans sens |
| **LLM seul** | Comprend le contexte | Peut halluciner des FDs |
| **Hybride** | Précis ET significatif | Meilleur des deux mondes |

### Ce qu'on a appris

1. **Les algorithmes trouvent TOUTES les FDs** valides techniquement, mais beaucoup sont triviales ou accidentelles

2. **Le LLM filtre par le sens** : il distingue les clés primaires, les règles métier, et les corrélations accidentelles

3. **Le score hybride** combine les deux pour identifier les FDs vraiment utiles

### Recommandation finale

> **Utilisez toujours une approche hybride** : les algorithmes pour la précision, les LLMs pour la pertinence.

In [ ]:
# Résumé final
print("\n" + "="*80)
print("RÉSUMÉ FINAL - TASK 4 : PIPELINE HYBRIDE")
print("="*80)

print(f"\n📊 Statistiques globales:")
print(f"   - Datasets analysés: {len(all_discovered_fds)}")
print(f"   - FDs évaluées: {len(results_df)}")
print(f"   - FDs significatives (score >= 7): {len(results_df[results_df['hybrid_score'] >= 7])}")
print(f"   - FDs utiles (5-7): {len(results_df[(results_df['hybrid_score'] >= 5) & (results_df['hybrid_score'] < 7)])}")
print(f"   - FDs à ignorer (< 5): {len(results_df[results_df['hybrid_score'] < 5])}")

print(f"\n📈 Score hybride moyen par catégorie:")
for cat, score in results_df.groupby('category')['hybrid_score'].mean().sort_values(ascending=False).items():
    print(f"   - {cat}: {score:.1f}")

print(f"\n📁 Fichiers générés:")
print(f"   - results/task4_hybrid_results.csv")
print(f"   - results/task4_hybrid_results.json")
print(f"   - results/task4_hybrid_analysis.png")

print("\n" + "="*80)
print("✅ PIPELINE HYBRIDE TERMINÉ AVEC SUCCÈS !")
print("="*80)